### Epmwebapi Quickstart 

**Connecting to EPM Webserver**

In [1]:
import epmwebapi as epm
import numpy as np
import datetime

user = 'user'
password = 'psw'

#The line below use the EpmConnection method to create a connection with a EPM Server
connection = epm.EpmConnection('http://machine_name:44333', 'http://machine_name:44332', user, password)


EPM user:sa
EPM password:········


**Reading raw data**

In [2]:
#basic variable name
bvname = 'ADM_Temperature'

#method to get the basic variable
bv = connection.getDataObjects([bvname])

#define query period
end_date = datetime.datetime.now()
ini_date= end_date - datetime.timedelta(minutes=10)
queryPeriod = epm.QueryPeriod(ini_date,end_date)

#make read
data = bv[bvname].historyReadRaw(queryPeriod)

#show results
print(data)


[]


**Reading aggregated data(OPCUA based)**

In [3]:
#basic variable name
bvname = 'ADM_Temperature'

#method to get the basic variable
bv = connection.getDataObjects([bvname])

#define query period
end_date = datetime.datetime.now()
ini_date= end_date - datetime.timedelta(hours=1)
queryPeriod = epm.QueryPeriod(ini_date,end_date)

#define aggregate details
processInterval = datetime.timedelta(minutes=10)
aggDetails = epm.AggregateDetails(processInterval, epm.AggregateType.Interpolative)

#make read
data = bv[bvname].historyReadAggregate(aggDetails, queryPeriod)

#show results
print(data)

[(0., datetime.datetime(2019, 12, 11, 15, 37, 59, 554738, tzinfo=datetime.timezone.utc), 2157641728)
 (0., datetime.datetime(2019, 12, 11, 15, 47, 59, 554738, tzinfo=datetime.timezone.utc), 2157641728)
 (0., datetime.datetime(2019, 12, 11, 15, 57, 59, 554738, tzinfo=datetime.timezone.utc), 2157641728)
 (0., datetime.datetime(2019, 12, 11, 16, 7, 59, 554738, tzinfo=datetime.timezone.utc), 2157641728)
 (0., datetime.datetime(2019, 12, 11, 16, 17, 59, 554738, tzinfo=datetime.timezone.utc), 2157641728)
 (0., datetime.datetime(2019, 12, 11, 16, 27, 59, 554738, tzinfo=datetime.timezone.utc), 2157641728)]


**Writing historical data to EPM Server**


In [ ]:
#Basic Variable name that receives data
bvname = 'bvname'

#method to get the basic variable
bv = connection.getDataObjects([bvname])

#some values for write tests
newvalues = [50,60,30,40,10]
base = datetime.datetime(2018, 1, 1)
newdates = np.array([base + datetime.timedelta(hours=i) for i in range(5)])

# epm ndarray data format
desc = np.dtype([('Value', '>f8'), ('Timestamp', 'object'), ('Quality', '>i4')])
datatemp = np.empty(len(newvalues), dtype=desc)

#loop to populate the object before send to EPM
i=0
while i < len(newvalues):
    datatemp['Value'][i] = newvalues[i]
    datatemp['Timestamp'][i] = newdates[i]
    datatemp['Quality'][i] = 0
    i = i+1
#make write    
bv[bvname].historyUpdate(datatemp)

**Write data to EPM Server** 

Sends value in real-time way. 

In [ ]:
bvname = 'bvname'

bv = connection.getDataObjects([bvname])

date = datetime.datetime.now()
value = 100
quality = 0 #zero is Good in OPC UA

bv[bvname].write(value, date, quality)


**Read the real-time value of a variable**

In [ ]:
bvname = 'ADM_Temperature'

bv = connection.getDataObjects(bvname)

data = bv[bvname].read()

print(data.value)
print(data.timestamp)

**Variable Filter**

Example of read variables that contains "Temperature" in the name and "Continuous" Type.

In [ ]:
from epmwebapi.dataobjectattributes import DataObjectAttributes
from epmwebapi.dataobjectsfilter import DataObjectsFilter
from epmwebapi.dataobjectsfilter import DataObjectsFilterType
from epmwebapi.domainfilter import DomainFilter

Filter = DataObjectsFilter(DataObjectsFilterType.BasicVariable, '*Temperature', None, None, DomainFilter.Continuous)

properties = DataObjectAttributes.Description | DataObjectAttributes.LowLimit | DataObjectAttributes.HighLimit | DataObjectAttributes.EU

query = connection.filterDataObjects(Filter, properties)


for item in list(query.values()):
    
    data = item.read()
    print('{} >>> {}'.format(item.name,data.value))   


**Browsing DataModel Structure** 

![ECC DataModel](./images/datamodel_elipse.png)


In [ ]:
#EPM Path that contains DataModel
basePath = '/Models/ElipseDataModel/DataModel/Elipse'

#Get address in EPM Server
itens = connection.getObjects(basePath)

#loop to find itens
for item in list(itens.values()):    
    
    #List the first level
    filiais = list(item.enumObjects().values())
    
    #list the second level
    for filial in filiais:        
        
        #list the second level and choose the second property (index 1), called Description.
        obj = list(filial.enumProperties().values())[1]
        
        # make read
        data = obj.read()
        #show the result
        print(data.value)

**writing annotations**

Annotation is a text message tha can be put on the timeline of a variable. In this example we write an annotation in the Max value of the variable. 

In [ ]:
bvname = 'ADM_Temperature'

bv = connection.getDataObjects([bvname])

end_date = datetime.datetime.now()
ini_date= end_date - datetime.timedelta(minutes=10)
queryPeriod = epm.QueryPeriod(ini_date,end_date)
data = bv[bvname].historyReadRaw(queryPeriod)

max_value = data['Value'].max()

max_index = np.argwhere(data['Value'] == max_value)

print(max_index)

print(data['Timestamp'][max_index][0])

#escrevendo anotação
message = 'valor maximo encontrado no periodo'
timestamp = data['Timestamp'][max_index]
bv[bvname].writeAnnotation(timestamp,message)

**Querying Annotations**

In [1]:
bvname = 'ADM_Temperature'

bv = connection.getDataObjects([bvname])

end_date = datetime.date(2018, 10, 19)
ini_date = end_date - datetime.timedelta(hours=24)

annotations = bv[bvname].readAnnotations(ini_date,end_date)
print(annotations)

NameError: name 'connection' is not defined

**Closing Connection**

We need to close connection, to release it to other possible users.

In [ ]:
connection.close()